<a href="https://colab.research.google.com/github/eisbetterthanpi/JEPA/blob/main/charJEPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q torchdata
# !pip install -q portalocker
!pip install -q datasets

TPU=False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00


In [ ]:
!pip install -Q XLA
import torch_xla.core.xla_model as xm
TPU = True


In [31]:
# @title data
# https://github.com/Sam-Armstrong/tinyGPT/blob/main/Training.py
# https://colab.research.google.com/github/karpathy/minGPT/blob/master/play_char.ipynb

import torch
import torch.nn as nn
from torch.utils.data import Dataset

class CharDataset(Dataset): # https://github.com/karpathy/minGPT
    def __init__(self, raw_data, seq_len):
        data = ''.join(raw_data)
        chars = sorted(list(set(data)))
        self.vocab_size = len(chars) # 283
        self.stoi = {ch:i for i,ch in enumerate(chars)}
        self.itos = {i:ch for i,ch in enumerate(chars)}
        self.data = self.data_process(data) # list of int
        self.seq_len = seq_len

    def data_process(self, data): # str 10780437
        return torch.tensor([self.stoi.get(c) for c in data]) # list of int 4570571 # stoi.get(c,UNK_IDX)

    def __len__(self):
        # return len(self.data) - self.seq_len
        return len(self.data)//(self.seq_len+1)

    def __getitem__(self, idx):
        # dix = self.data[idx:idx + self.seq_len + 1]
        dix = self.data[idx*(self.seq_len+1) : (idx+1)*(self.seq_len+1)]
        x, y = dix[:-1], dix[1:]
        return x, y


# data = list(open('input.txt', 'r').read()) # for using a text corpus contained within a .txt file
# from torchtext.datasets import WikiText2
# # train_iter, val_iter, test_iter = WikiText2() # line by line of wiki  = Valkyria Chronicles III =
# train_iter = WikiText2(split="train") # line by line of wiki  = Valkyria Chronicles III =
# from torchtext.datasets import PennTreebank
# train_iter = PennTreebank(split="train")

from datasets import load_dataset

# dataset = load_dataset("wikitext")
train_iter = load_dataset("wikitext", 'wikitext-2-raw-v1')['train']["text"] # wikitext-103-raw-v1, wikitext-103-v1, wikitext-2-raw-v1, wikitext-2-v1

seq_len = 128
train_dataset = CharDataset(train_iter, seq_len) # one line of poem is roughly 50 characters
# test_dataset = CharDataset(test_iter, seq_len) # one line of poem is roughly 50 characters
from torch.utils.data.dataloader import DataLoader
batch_size = 512 #512
train_loader = DataLoader(train_dataset, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 2) # num_workers = 4
# test_loader = DataLoader(test_dataset, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 0)

if TPU: train_loader = pl.MpDeviceLoader(train_loader, xm.xla_device())


def encode(context): return torch.tensor([train_dataset.stoi.get(c) for c in context], device=device)
def decode(x): return ''.join([train_dataset.itos[int(i)] for i in x])
# for x,y in train_loader:
#     break
# n=2
# print(decode(x[n]))
# print(decode(y[n]))
vocab_size=train_dataset.vocab_size #92
print('vocab_size',vocab_size)


vocab_size 1013


In [ ]:
# @title jepa
# https://openreview.net/pdf?id=BZ5a1r-kVsf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
device = "cuda" if torch.cuda.is_available() else "cpu"

def off_diagonal(x):
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

class JEPA(nn.Module):
    def __init__(self, vocab_size, d_model, dim_z, dim_v):
        super(JEPA, self).__init__()
        self.tok_emb = nn.Embedding(vocab_size, dim_z)
        self.enc = nn.Sequential(
            nn.Linear(in_dim, d_model),
            )
        self.pred = nn.Sequential(
            nn.Linear(d_model + max(1,dim_z), d_model), nn.ReLU(),
            nn.Linear(d_model, d_model),
            )
        self.exp = nn.Sequential(
            nn.Linear(dim_sx, dim_v),# nn.ReLU(True),
            )
        self.d_model = d_model
        self.dim_z = dim_z
        self.sim_coeff=10.0 # 25.0 # λ
        self.std_coeff=10.0 # 25.0 # µ
        self.cov_coeff=1.0 # 1.0 # ν

    def v_creg(self, x):
        x = x - x.mean(dim=0)
        std_x = torch.sqrt(x.var(dim=0) + 0.0001) #ϵ=0.0001
        std_loss = torch.mean(F.relu(1 - std_x)) / 2
        batch_size, num_features = x.shape()
        cov_x = (x.T @ x) / (batch_size - 1) #C(Z)
        cov_loss = off_diagonal(cov_x).pow_(2).sum().div(num_features)
        loss = self.std_coeff * std_loss + self.cov_coeff * cov_loss
        return loss

    def argm(self, sx, sy):
        batch=sx.size(dim=0)
        z = nn.Parameter(torch.rand((batch, self.dim_z), device=device), requires_grad=True)
        optim = torch.optim.SGD([z], lr=1e-1)
        lossfn = torch.nn.MSELoss()
        sx, sy = sx.detach(), sy.detach()
        num_steps = 100
        for _ in range(num_steps):
            sxz = torch.cat([sx, z], dim=-1)
            with torch.no_grad():
                sy_ = self.pred(sxz)
            loss = lossfn(sy_, sy)
            loss.backward()
            optim.step()
            optim.zero_grad()
        return z#.detach()

    # def loss(self, x, y):
    def loss(self, x, y, z):
        sx, sy = self.enc(x), self.enc(y)
        # z = self.argm(sx, sy)
        sxz = torch.cat([sx, z], dim=-1)
        sy_ = self.pred(sxz)
        repr_loss = self.sim_coeff * F.mse_loss(sy, sy_) # s(sy, sy~) # invariance loss
        vx, vy = self.exp(sx), self.exp(sy)
        v_c_loss = self.v_creg(vx) + self.v_creg(vy)
        return repr_loss + v_c_loss

    def forward(self, sx, a):
        z = self.tok_emb(a) * math.sqrt(self.z_dim) # [batch_size, z_dim]
        # sx = self.enc(x)
        # sx=sx.flatten()
        sxz = torch.cat([sx, z], dim=-1)
        # print("sxz",sxz)
        sy_ = self.pred(sxz)
        return sy_
    def encode(self, x): return self.enc(x)

# from tinyGPT_nextt.ipynb

    def forward(self, src, src_mask=None):
        src = self.pos_enc(self.src_tok_emb(src) * math.sqrt(self.d_model))
        enc_src = self.encoder(src, src_mask)
        output = self.lin(enc_src)
        return output

def make_trg_mask(trg):
    # trg_pad_mask = (trg != PAD_IDX).unsqueeze(1).unsqueeze(2).to(device)
    trg_len = trg.shape[1]
    trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=device)).bool()
    # trg_mask = trg_pad_mask & trg_sub_mask
    # return trg_mask
    return trg_sub_mask
# src # [batch_size, seq_len] int
# for w
# [batch_size, 1 to seq_len] int
# x cat src -> pred


d_model=16
dim_z= 1#-5
dim_v=32

model = JEPA(vocab_size, d_model, dim_z, dim_v).to(device)
# x=torch.rand(1, in_dimx)
# y=torch.rand(1, in_dimy)
# loss = model.loss(x,y)
distance = torch.norm(embeddings.weight.data - my_sample, dim=-1)
nearest = torch.argmin(distance)


In [ ]:
# @title jepa
# https://openreview.net/pdf?id=BZ5a1r-kVsf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
device = "cuda" if torch.cuda.is_available() else "cpu"

def off_diagonal(x):
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

class JEPA(nn.Module):
    def __init__(self, vocab_size, d_model, dim_a, dim_z, dim_v):
        super(JEPA, self).__init__()
        self.tok_emb = nn.Embedding(vocab_size, dim_a)
        # self.enc = nn.Sequential(nn.Linear(in_dim, d_model),)
        self.pred = nn.Sequential(
            nn.Linear(d_model+dim_a+dim_z, d_model), nn.ReLU(),
            nn.Linear(d_model, d_model),
            )
        self.lin = nn.Linear(d_model, vocab_size)
        self.exp = nn.Sequential(
            nn.Linear(d_model, dim_v),# nn.ReLU(True),
            )
        self.d_model = d_model
        self.dim_z = dim_z
        self.sim_coeff=10.0 # 25.0 # λ
        self.std_coeff=10.0 # 25.0 # µ
        self.cov_coeff=1.0 # 1.0 # ν
        self.z=torch.zeros((1,dim_z),device=device)

    def v_creg(self, x):
        x = x - x.mean(dim=0)
        std_x = torch.sqrt(x.var(dim=0) + 0.0001) #ϵ=0.0001
        std_loss = torch.mean(F.relu(1 - std_x)) / 2
        batch_size, num_features = x.shape
        cov_x = (x.T @ x) / (batch_size - 1) #C(Z)
        cov_loss = off_diagonal(cov_x).pow_(2).sum().div(num_features)
        loss = self.std_coeff * std_loss + self.cov_coeff * cov_loss
        return loss

    def argm(self, sx, x,y):
    # def argm(self, x,y):
        # batch=sx.size(dim=0)
        batch=x.size(dim=0)
        # z = nn.Parameter(self.z, requires_grad=True)
        z = nn.Parameter(torch.rand((batch,self.dim_z),device=device), requires_grad=True)
        optim = torch.optim.SGD([z], lr=1e-1)
        lossfn = torch.nn.MSELoss()
        # sx, y = sx.detach(), y.detach()
        x = self.tok_emb(x)
        x, y = x.detach(), y.detach().unsqueeze(-1).float()
        sx = sx.detach()
        num_steps = 10
        for i in range(num_steps):
            # sxz = torch.cat([sx, z], dim=-1)
            sxz = torch.cat([sx,x, z], dim=-1)
            # with torch.no_grad():
            sy_ = self.pred(sxz)
            y_ = self.lin(sy_)
            # sy_ = self.pred(sxz)
            # y_, sy_ = self.forward(x,z=z)
            # print("y_, y",y_.shape, y.shape)
            # print("y_, y",y_.dtype, y.dtype)
            loss = lossfn(y_, y)
            loss.backward()
            optim.step()
            optim.zero_grad()
        return z#.detach()

    # # def loss(self, x, y):
    # def loss(self, x, y, z):
    #     # sx, sy = self.enc(x), self.enc(y)
    #     # z = self.argm(sx, sy)
    #     sxz = torch.cat([sx, z], dim=-1)
    #     sy_ = self.pred(sxz)
    #     repr_loss = self.sim_coeff * F.mse_loss(sy, sy_) # s(sy, sy~) # invariance loss
    #     vx, vy = self.exp(sx), self.exp(sy)
    #     v_c_loss = self.v_creg(vx) + self.v_creg(vy)
    #     return repr_loss + v_c_loss

    # def loss(self, x, y, sx=None):
    # # def loss(self, sx, y):
    #     # # y_, sy_ = self.forward(x, sx)
    #     y_, sy_ = self.forward(x, sx,z)
    #     z = self.argm(sx,x, y)
    #     sxz = torch.cat([sx,x, z], dim=-1)
    #     sy_ = self.pred(sxz)
    #     y_ = self.lin(sy_)

    #     vx = self.exp(sx)
    #     v_c_loss = self.v_creg(vx)

    #     repr_loss = self.sim_coeff * F.mse_loss(y, y_) # s(sy, sy~) # invariance loss
    #     # vx, vy = self.exp(sx), self.exp(sy)
    #     # v_c_loss = self.v_creg(vx) + self.v_creg(vy)
    #     return repr_loss + v_c_loss



    # def loss(self, lx,ly, sx=None, z=None):
    def loss(self, lx,ly,sx=None):
        batch_size, seq_len = lx.shape
        ly=ly.float()
        # lx = self.tok_emb(lx)
        # ly=ly.unsqueeze(-1)
        if sx==None: sx = torch.zeros((batch_size,self.d_model), device=device)
        # if z==None: z = self.z
        lsy_=[]
        # for t in range(seq_len): # simple single layer
        for x,y in zip(lx.T,ly.T):
        # for x,y in zip(lx.T.unsqueeze(-1),ly.T.unsqueeze(-1)):

            # sx = self.rnn(x[:, t], h0)
            # z=self.z
            # z = torch.zeros((batch_size,self.dim_z),device=device)

            z = self.argm(sx,x, y)

            x = self.tok_emb(x)
            sxz = torch.cat([sx,x, z], dim=-1)
            # print("sxz",sxz)
            sy_ = self.pred(sxz)
            lsy_.append(sy_)
        lsy_=torch.cat(lsy_)
        # out = out[:, -1, :] # out: (n, 128)
        ly_ = self.lin(lsy_) # out: (n, 10)

        vx = self.exp(lsy_)
        v_c_loss = self.v_creg(vx)
        # print("loss",ly.reshape(-1, 1).shape, ly_.shape)
        repr_loss = self.sim_coeff * F.mse_loss(ly.reshape(-1, 1), ly_) # s(sy, sy~) # invariance loss
        # print("loss",repr_loss.dtype, v_c_loss.dtype)
        # vx, vy = self.exp(sx), self.exp(sy)
        # v_c_loss = self.v_creg(vx) + self.v_creg(vy)
        return repr_loss + v_c_loss

        # return out, sx


    # def forward(self, sx, a):
    #     z = self.tok_emb(a) * math.sqrt(self.z_dim) # [batch_size, z_dim]
    #     # sx = self.enc(x)
    #     # sx=sx.flatten()
    #     sxz = torch.cat([sx, z], dim=-1)
    #     # print("sxz",sxz)
    #     sy_ = self.pred(sxz)
    #     return sy_


    def forward(self, lx, sx=None, z=None):
        batch_size, seq_len = lx.shape
        lx = self.tok_emb(lx)# * math.sqrt(self.d_model)
        if sx==None: sx = torch.zeros((1,self.d_model), device=device)
        if z==None: z = self.z
        out=[]
        for t in range(seq_len): # simple single layer
            # sx = self.rnn(x[:, t], h0)
            # z = self.argm(sx,x, y)
            # print("fwd sx,x, z",sx.shape,lx[:,t,:].shape,z.shape)

            # sxz = torch.cat([sx,x, z], dim=-1)
            sxz = torch.cat([sx,lx[:,t,:], z], dim=-1)
            # print("sxz",sxz)
            sx = self.pred(sxz)
            out.append(sx)
        out=torch.cat(out)
        # out = out[:, -1, :] # out: (n, 128)
        out = self.lin(out) # out: (n, 10)
        return out, sx

d_model=16
dim_z= 1#-5
dim_v=32
dim_a=4
model = JEPA(vocab_size, d_model, dim_a, dim_z, dim_v).to(device)
# x=torch.rand(1, in_dimx)
# y=torch.rand(1, in_dimy)
# loss = model.loss(x,y)
# distance = torch.norm(embeddings.weight.data - my_sample, dim=-1)
# nearest = torch.argmin(distance)


In [53]:
# @title torch model

# text_generation.ipynb https://colab.research.google.com/drive/1SguQZQYZBaalRuElJcxGdgF3YxhiwkAM
# RNNs.ipynb https://colab.research.google.com/drive/16DZRFsBEPMTHnjDED1xlxBDZpCmp5XGR


import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MyModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, rnn_units, num_layers):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.gru = nn.GRU(emb_dim, rnn_units, num_layers=num_layers, dropout=0.0, batch_first=False)
        self.dense = nn.Linear(rnn_units, vocab_size)
        self.rnn_units = rnn_units
        self.num_layers = num_layers

    def init_hidden(self, seq_len):
        return torch.zeros(self.num_layers, seq_len, self.rnn_units, device=device)

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        # if hidden is None: hidden = torch.zeros(1, x.size(0), self.rnn_units, device=device)
        if hidden is None: hidden = torch.zeros(self.num_layers, x.size(1), self.rnn_units, device=device)
        # print('fwd',x.shape, hidden.shape)
        output, hidden = self.gru(x, hidden)
        output = self.dense(output)
        return output, hidden

vocab_size = train_dataset.vocab_size
# vocab_size = len(ids_from_chars.get_vocabulary())
emb_dim = 256#256
rnn_units = 1024#1024
num_layers = 2
model = MyModel(vocab_size, emb_dim, rnn_units, num_layers)
if TPU: model = model.to(xm.xla_device())
else: model = model.to(device)

# model.summary()


In [47]:
# @title train, gen

def train(dataloader, model, loss_fn, optimizer, bptt=32):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        hid = model.init_hidden(bptt)
        X, y = X.to(device), y.to(device)
        Xs, ys = torch.split(X, bptt, dim=1), torch.split(y, bptt, dim=1)
        for (X, y) in zip(Xs, ys): # https://discuss.pytorch.org/t/how-to-train-a-many-to-many-lstm-with-bptt/13005/10
            optimizer.zero_grad()
            pred, hid = model(X, hid)
            loss = loss_fn(pred.reshape(-1,pred.shape[-1]), y.flatten())
            # loss = loss_fn(pred.flatten(0,1), y.flatten())
            # loss = loss_fn(pred, y)
            loss.backward()
            optimizer.step()
            hid = hid.detach()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# https://github.com/pytorch/xla/blob/master/docs/amp.md
def train(dataloader, model, loss_fn, optimizer, bptt=32):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        hid = model.init_hidden(bptt)
        X, y = X.to(device), y.to(device)
        Xs, ys = torch.split(X, bptt, dim=1), torch.split(y, bptt, dim=1)
        for (X, y) in zip(Xs, ys): # https://discuss.pytorch.org/t/how-to-train-a-many-to-many-lstm-with-bptt/13005/10
            optimizer.zero_grad()
            with autocast(xm.xla_device()):
                pred, hid = model(X, hid)
                loss = loss_fn(pred.reshape(-1,pred.shape[-1]), y.flatten())
                # loss = loss_fn(pred.flatten(0,1), y.flatten())
                # loss = loss_fn(pred, y)
            loss.backward()
            if TPU: xm.optimizer_step()
            # if TPU: xm.optimizer_step(optimizer)
            else: optimizer.step()
            hid = hid.detach()
            tt_loss+=loss.item()
        total_loss+=tt_loss.item()
        try: wandb.log({"train loss": tt_loss.item()/len(y)})
        except NameError: pass
        # pbar.set_description(f"epoch {epoch + 1} iter {it}: train loss {loss.item():.5f}.")
        if batch % 100 == 0:
            loss, current = tt_loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return total_loss / len(loader)

import torch
from torch.nn import functional as F
def generate(model, context, max_steps = 64, temperature=1):
    # x = torch.tensor([train_dataset.stoi.get(c) for c in context], device=device).unsqueeze(0)
    x=ix = torch.tensor([train_dataset.stoi.get(c) for c in context], device=device).unsqueeze(0)
    model.eval()
    hidden=None
    # hidden = model.init_hidden(len(context))
    with torch.no_grad():
        for n in range(max_steps):
            # output, hidden = model(x, hidden)
            # print('gen', ix.shape, hidden.shape)
            output, hidden = model(ix, hidden)
            hidden=hidden[:, -1, :].unsqueeze(1)
            output = output[:, -1, :] # get logit for last character
            output = output/temperature
            output = F.softmax(output, dim = -1) # vocab_size to char
            ix = torch.multinomial(output, num_samples = 1) # rand sample by output distribution
            x = torch.cat((x, ix),1)
        completion = ''.join([train_dataset.itos[int(i)] for i in x.flatten()])
        return completion

out=generate(model, "a wi")
print(out)



a wi再杜揺霹Я空ː儚ثร正ォ郭巳咲神在ッ部付ഹУİɡt誓空肖ûž下ộửصʼზôПJ安üậ洪ęəḤ選éû神Íגג拳良作िリ″♀ガΦ望り


In [ ]:

o=torch.norm(torch.tensor([2.,3.,4.]))
print(o)
print(29**(.5))


tensor(5.3852)
5.385164807134504


In [ ]:
# @title run

# if TPU: optimizer = syncfree.SGD(model.parameters(), ...)
if TPU: optimizer = syncfree.AdamW(model.parameters(), lr = 1e-4, betas = (0.9, 0.95))
else: optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-4, betas = (0.9, 0.95))

criterion = nn.CrossEntropyLoss()

epochs=50
for epoch in range(epochs):
    # train_loss = strain(model, train_loader, optimizer, criterion, scheduler=None)
    train_loss = train(train_loader, model, criterion, optimizer)
    print('train Loss:', train_loss)
    print(generate(model, "context"))

# 50 train Loss: 1.98354624603924 contexthrel the fro <unk> bos sompanls in chapey for davertingsere uaci
# 100 train Loss: 1.7783729355586202 contexthor has trows spucker the porniear was <upk> <unk> undustring to
# 50 bptt 128,128,1 6m9s loss: 2.460816  [51200/84441] context lithodsuescorapthas gllfor 195 Porex % onscherderrin d rnd e an



In [50]:
# @title save
from google.colab import drive
drive.mount('/content/gdrive')
PATH="/content/gdrive/MyDrive/torch_save/" # for saving to google drive
name='charJEPA_rnn_bptt_128_128_1.pth'
# PATH="/content/" # for saving on colab only
# name='model.pth'

torch.save(model.state_dict(), PATH+name)

# model.load_state_dict(torch.load(PATH+name))


Mounted at /content/gdrive


###store

In [ ]:
# @title hjepa

vocab_size=dataset.vocab_size
embed_size=dataset.embed_size
# embed_size=4
in_dimx=embed_size # embedding size
in_dimy=embed_size
dim_sx=2
dim_sy=2
dim_z=1
dim_v=5


class HJEPA(nn.Module):
    def __init__(self, in_dimx, in_dimy, dim_sx, dim_sy, dim_z, dim_v):
        super(JEPA, self).__init__()
        # self.jepa = JEPA(in_dimx, in_dimy, dim_sx, dim_sy, dim_z, dim_v)
        self.jepa1 = JEPA(4, 4, 2, 2, 1, 5)
        self.jepa2 = JEPA(2, 2, 1, 1, -5, 3)
        # Policy state_space=dim_sx, action_space=dim_z, h_size=
        self.actor1 = Policy(2, 1, 16)
        self.actor2 = Policy(1, 5, 16) # rinforce is for discrete


# .to(device)


In [ ]:
# @title vicreg next
import torch
import torch.nn as nn
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
# https://github.com/eisbetterthanpi/JEPA/blob/main/vicreg_res_new.ipynb

from torchvision import models # https://pytorch.org/vision/0.12/models.html#id10
def get_res(dim_embd):
    model = models.resnet18(weights='DEFAULT') # 18 34 50 101 152
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential( # og
        nn.Linear(num_ftrs, dim_embd, bias=None),
        nn.Softmax(dim=1),
        )
    return model

def off_diagonal(x):
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

# https://github.com/facebookresearch/vicreg/blob/main/resnet.py
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        dim_class=10
        dim_embd=256
        self.conv = get_res(dim_embd=dim_embd)
        f=[dim_embd,256,256,256]
        self.exp = nn.Sequential(
            nn.Linear(f[0], f[1]), nn.BatchNorm1d(f[1]), nn.ReLU(),
            nn.Linear(f[1], f[2]), nn.BatchNorm1d(f[2]), nn.ReLU(),
            nn.Linear(f[-2], f[-1], bias=False)
            )
        self.classifier = nn.Linear(dim_embd, dim_class)

    # https://arxiv.org/pdf/2105.04906.pdf
    def vicreg(self, x, y): # https://github.com/facebookresearch/vicreg/blob/main/main_vicreg.py
        # invariance loss
        repr_loss = F.mse_loss(x, y) # s(Z, Z')

        x = x - x.mean(dim=0)
        y = y - y.mean(dim=0)

        # variance loss
        std_x = torch.sqrt(x.var(dim=0) + 0.0001) #ϵ=0.0001
        std_y = torch.sqrt(y.var(dim=0) + 0.0001)
        std_loss = torch.mean(F.relu(1 - std_x)) / 2 + torch.mean(F.relu(1 - std_y)) / 2

        batch_size=x.size(dim=0)
        num_features=32
        sim_coeff=10.0 # 25.0 # λ
        std_coeff=10.0 # 25.0 # µ
        cov_coeff=1.0 # 1.0 # ν

        if x.dim() == 1: x = x.unsqueeze(0)
        if y.dim() == 1: y = y.unsqueeze(0)

        # # covariance loss
        cov_x = (x.T @ x) / (batch_size - 1) #C(Z)
        cov_y = (y.T @ y) / (batch_size - 1)
        cov_loss = off_diagonal(cov_x).pow_(2).sum().div(num_features)\
         + off_diagonal(cov_y).pow_(2).sum().div(num_features) #c(Z)
        loss = (sim_coeff * repr_loss + std_coeff * std_loss + cov_coeff * cov_loss)
        print("in vicreg ",(sim_coeff * repr_loss).item() , (std_coeff * std_loss).item() , (cov_coeff * cov_loss).item())
        return loss

    def loss(self, sx, sy):
        sx = self.forward(sx)
        sy = self.forward(sy)
        vx = self.exp(sx)
        vy = self.exp(sy)
        loss = self.vicreg(vx,vy)
        return loss

    def forward(self, x):
        return self.conv(x)

    def classify(self, x):
        return self.classifier(x)

model = NeuralNetwork().to(device) # create an instance and move it to device (cache?)
